Importing all necessary modules at the top of the file.

In [2]:
import pandas as pd
from IPython.display import HTML
import webbrowser
import os

The View function is designed to open a new browser window and open the Pandas dataframe as a HTML table, replicating the View function in R Studio. There is custom CSS styling included to provide a straightforward view of the dataFrame.

In [3]:
def view_sample(df, sample_size):
    css = """<style>
    table { border-collapse: collapse; border: 3px solid #eee; }
    table tr th:first-child { background-color: #eeeeee; color: #333; font-weight: bold }
    table thead th { background-color: #eee; color: #000; }
    tr, th, td { border: 1px solid #ccc; border-width: 1px 0 0 1px; border-collapse: collapse;
    padding: 3px; font-family: monospace; font-size: 10px }</style>
    """
    
    sample= df.sample(n=sample_size)
    html = sample.to_html(escape=False) + css
   
    path = 'temp.html'
    with open(path, 'w') as f:
        f.write(html)
    
    url = 'file://' + os.path.realpath(path) # Used to get the absolute path of the html page we've created
    webbrowser.open(url)
    
def view_all(df):
    css = """<style>
    table { border-collapse: collapse; border: 3px solid #eee; }
    table tr th:first-child { background-color: #eeeeee; color: #333; font-weight: bold }
    table thead th { background-color: #eee; color: #000; }
    tr, th, td { border: 1px solid #ccc; border-width: 1px 0 0 1px; border-collapse: collapse;
    padding: 3px; font-family: monospace; font-size: 10px }</style>
    """
    
    html = df.to_html(escape=False) + css
   
    path = 'temp.html'
    with open(path, 'w') as f:
        f.write(html)
    
    url = 'file://' + os.path.realpath(path) # Used to get the absolute path of the html page we've created
    webbrowser.open(url)

We are focusing on the National Center for Educational Studies' public elementary/secondary school universe survey in 2022-2023 and public school CCD geocode dataset. The datasets encaptures an extensive list of information of all the public schools in the United States. The goal is to extract the contact information and geolocation of all elementary and secondary schools across America to be readily accessible for users, ultimately providing the ability to reach out to nearby schools of any location in the US with corresponding email addresses.

Setting the dataframaes id (info_directory) and gd (geocode_directory).

In [4]:
id = pd.read_excel('data/ccd/ccd_public_school_directory.xlsm', engine='openpyxl')
gd = pd.read_excel('data/geocodes/public_school_geocodes.xlsx', engine='openpyxl')

id_docs_file = pd.ExcelFile('data/ccd/ccd_public_school_directory_companion.xlsx', engine='openpyxl')
id_docs = pd.read_excel(id_docs_file, sheet_name='File Layout')

We are isolating the variable names and its descriptions to better understand what these representations mean in the public school directories. The variables of interest are\
STATENAME: State name\
SCH_NAME: School name\
ST_SCHID: State school identifier\
NCESSCH: School Identifier (NCES)\
\
MSTREET1: Mailing address street 1\
MCITY: Mailing city\
MSTATE: Mailing state\
MZIP: Mailing 5 digit ZIP code\
LSTREET1: Location address street 1\
LCITY: Location city street 1\
LSTATE: Location state. Two-letter U.S. Postal Service abbreviation\
LZIP: Location 5 digit ZIP code\
\
PHONE: Telephone number\
WEBSITE: The URL for the unique address of a Web Page of an education entity\
SY_STATUS: Start of year status (1-Open, 2-Closed, 3-New, 4-Added, 5-Changed Boundary/Agency, 6-Inactive, 7-Future, 8-Reopened)\
UPDATED_STATUS: Updated Status (1-Open, 2-Closed, 3-New, 4-Added, 5-Changed Boundary/Agency, 6-Inactive, 7-Future, 8-Reopened)\
SCH_TYPE: School type (1-Regular School, 2-Special Education School, 3-Career and Technical School, 4-Alternative Education School)\
Level: School Level (Elementary, High, Other, Middle, Not reported, Not applicable, Ungraded, Adult Education, Prekindergarten, Secondary)\

In [5]:
len_id_docs = len(id_docs.columns)

indices_dropped = [0] + list(range(2, 6)) + list(range(7, len_id_docs))
clean_id_docs = id_docs.drop(id_docs.columns[indices_dropped], axis = 1)

clean_id = id[['STATENAME', 'SCH_NAME', 'ST_SCHID', 'NCESSCH', 'MSTREET1', 'MCITY', 'MSTATE', 'MZIP', 'LSTREET1', 'LCITY', 'LSTATE', 'LZIP', 'PHONE', 'WEBSITE', 'SY_STATUS', 'UPDATED_STATUS', 'SCH_TYPE', 'LEVEL']]

Within the gd dataset, the variables of interest are

NCESSCH: School identification number\
LAT: Latitude\
LON: Longitude\
CBSA: Core Based Statistical Area\
NMCBSA: Core Based Statistical Area Name\
CSA: Combined Statistical Area\
NMCSA: Combined Statistical Area Name

In [6]:
len_gd = len(gd.columns)

indices_dropped = list(range(1, 11)) + [15] + list(range(18, len_gd))
clean_gd = gd[['NCESSCH', 'LAT', 'LON', 'CBSA', 'NMCBSA', 'CSA', 'NMCSA']]

**Some considerations we want to think about**:

Do we want to also include variables that group schools together?\
For example, variables like LEAID, LEA_NAME, NMCSA, NMCBSA, CNTY, NMCNT group schools together through districts, counties, towns, etc. 
- These variables might be useful when optimizing the time complexity of returning the school email addresses based on time complexity

We are checking if the gd and id have the same number of schools in the dataset. They both have 102,268 rows, alluding that the dataset contains the same exact schools. This identicalness is perfect to join the datsets together for the most helpful variables to join together.

In [7]:
print(len(gd))
print(len(id))

102268
102268


We have created a combined dataset containing both school and gecode information, which is necessary to find their email addresses and specific locations relative to the uesr's whereabouts.

In [9]:
combined_df = pd.merge(clean_id, clean_gd, on='NCESSCH', how='inner')
print(len(combined_df))

combined_df.to_csv('schools_information_dataset.csv', index=False)

102268
